In [19]:
!python -m pip uninstall -y bitsandbytes
!python -m pip install -q --no-cache-dir "bitsandbytes>=0.45.0" "transformers>=4.44.2" "accelerate>=1.0.1"


Found existing installation: bitsandbytes 0.47.0
Uninstalling bitsandbytes-0.47.0:
  Successfully uninstalled bitsandbytes-0.47.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 261.5 MB/s eta 0:00:0000:0100:01


In [20]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Run DeepSeek (local Hugging Face checkpoint) on a dataset.

- Input:  test_250.xlsx with an `instruction` column
- Output: test_250_with_output.xlsx with new `output` column
"""

import os
import pandas as pd
import torch
#from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
)


# ---------------- SETTINGS ----------------
INPUT_FILE  = "/kaggle/input/deepseek-dataset/test_250.xlsx"
OUTPUT_FILE = "test_250_with_output.xlsx"

MODEL_ID = "deepseek-ai/deepseek-coder-6.7b-instruct"  # change if using another checkpoint
SYSTEM_PROMPT = "You are a Python tutor who always writes correct and runnable code without any other descriptions according to the given instructions."

TEMPERATURE = 0.3
TOP_P = 0.9
MAX_NEW_TOKENS = 512
DTYPE = "auto"   # "auto", "float16", "bfloat16", "float32"
# ------------------------------------------


# def load_model(model_id: str, dtype: str = "auto"):
#     dtype_map = {
#         "auto": None,
#         "float16": torch.float16,
#         "bfloat16": torch.bfloat16,
#         "float32": torch.float32,
#     }
#     torch_dtype = dtype_map.get(dtype, None)

#     tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_id,
#         device_map="auto",
#         torch_dtype=torch_dtype,
#         trust_remote_code=True,
#     )
#     return tokenizer, model


def _can_use_bnb_4bit() -> bool:
    try:
        import bitsandbytes as bnb  # noqa
        import torch
        if not torch.cuda.is_available():
            return False
        # Optional deeper check that the CUDA lib is actually found
        try:
            from bitsandbytes.cuda_setup.main import get_cuda_lib_handle
            return get_cuda_lib_handle() is not None
        except Exception:
            # If this import path changes, assume OK and let HF validator decide
            return True
    except Exception:
        return False

def load_model(model_id: str, dtype: str = "auto"):
    # Map dtype string
    dtype_map = {
        "auto": None,
        "float16": torch.float16,
        "bfloat16": torch.bfloat16,
        "float32": torch.float32,
    }
    torch_dtype = dtype_map.get(dtype, None)

    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    # Try bnb 4-bit first
    if _can_use_bnb_4bit():
        try:
            compute_dtype = torch.float16  # good default for T4
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=compute_dtype,
            )
            model = AutoModelForCausalLM.from_pretrained(
                model_id,
                quantization_config=bnb_config,
                device_map="auto",            # let Accelerate place layers
                trust_remote_code=True,
            )
            model.eval()
            if torch.cuda.is_available():
                torch.backends.cuda.matmul.allow_tf32 = True
            return tokenizer, model
        except ImportError as e:
            print("[WARN] bitsandbytes 4-bit unavailable:", e)
        except Exception as e:
            print("[WARN] 4-bit load failed, falling back to fp16/bf16:", e)

    # Fallback: no bnb or 4-bit failed → load normally
    # Prefer fp16 on GPU; bf16 is fine on newer GPUs; float32 on CPU.
    if torch.cuda.is_available():
        fallback_dtype = torch.float16 if torch_dtype is None else torch_dtype
    else:
        # CPU has no half types
        fallback_dtype = torch.float32 if torch_dtype in (None, torch.float16, torch.bfloat16) else torch_dtype

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=fallback_dtype,
        trust_remote_code=True,
    )
    model.eval()
    return tokenizer, model



def apply_chat_template(tokenizer, system_prompt: str, user_prompt: str) -> str:
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    if hasattr(tokenizer, "apply_chat_template") and tokenizer.chat_template:
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return f"<system>{system_prompt}</system>\n<user>{user_prompt}</user>\n<assistant>"


def generate_response(tokenizer, model, user_prompt: str) -> str:
    prompt = apply_chat_template(tokenizer, SYSTEM_PROMPT, user_prompt)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            do_sample=True if TEMPERATURE > 0 else False,
            temperature=TEMPERATURE,
            top_p=TOP_P,
            max_new_tokens=MAX_NEW_TOKENS,
            eos_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out_ids[0][inputs["input_ids"].shape[1]:],
                            skip_special_tokens=True).strip()


def main():
    # Load model
    tokenizer, model = load_model(MODEL_ID, DTYPE)

    # Load dataset
    df = pd.read_excel(INPUT_FILE)
    if "instruction" not in df.columns:
        raise ValueError("File must contain an `instruction` column.")

    # Generate outputs
    outputs = []
    for i, instr in enumerate(df["instruction"]):
        if pd.isna(instr) or not str(instr).strip():
            outputs.append("")
            continue
        print(f"[{i+1}/{len(df)}] Generating for: {str(instr)[:50]}...")
        ans = generate_response(tokenizer, model, str(instr))
        outputs.append(ans)

    df["output"] = outputs

    # Save
    df.to_excel(OUTPUT_FILE, index=False)
    print(f"[DONE] Results saved -> {OUTPUT_FILE}")


if __name__ == "__main__":
    main()


[WARN] bitsandbytes 4-bit unavailable: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

[248/250] Generating for: ল্যাম্বদা ফাংশন ব্যবহার করে প্রদত্ত তালিকায় অদ্ভু...


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


[249/250] Generating for: একটি ফাংশন লিখুন যা দুইটি টুপলকে সর্বোচ্চ করে।
Exa...


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


[250/250] Generating for: nth নিউম্যান  শ্যাঙ্কস  উইলিয়ামস প্রাইম সংখ্যা খু...
[DONE] Results saved -> test_250_with_output.xlsx
